In [43]:
import torch
from torch import nn 
from torch.nn import functional as F

In [2]:
x = torch.randn(2, requires_grad=True) # requires_grad=True를 해야 gradient를 저장하여 사용 가능
y = x*3
gradients = torch.tensor([100, 0.1], dtype=torch.float)
y.backward(gradients)
print(x.grad)# (dy/dx)*gradient 출력
# 굳이 주어진 gradients값이 곱해 출력하는 이유는, gradient가 또다른 gradient일 때, chain rule을 만족시키기 위해
# 필요없다면 1을 넣어주면 된다.

tensor([300.0000,   0.3000])


In [5]:
# requires_grad=False의 예제
x = torch.randn(2, requires_grad=False) # requires_grad=True를 해야 gradient를 저장하여 사용 가능
y = x*3
gradients = torch.tensor([100, 0.1], dtype=torch.float)
y.backward(gradients)
print(x.grad)

RuntimeError: ignored

In [9]:
x = torch.randn(2, requires_grad=True)
y = x*3
gradients = torch.tensor([100, 0.1], dtype=torch.float)
y.backward(gradients, retain_graph= False) # False 일 경우 ErrorThrow
print(x.grad) 
y.backward(gradients) # 2번 미분해야하는 경우 retain_graph=True, 번 호출 시, 미분 값이 더해져 중첩됨 
print(x.grad)

tensor([300.0000,   0.3000])


RuntimeError: ignored

In [8]:
x = torch.randn(2, requires_grad=True)
y = x*3
gradients = torch.tensor([100, 0.1], dtype=torch.float)
y.backward(gradients, retain_graph= True) # retain_graph=True가 아니면, backward하고 난 뒤, 최적화를 위해 graph를 지워버림(종말단의 gradient만 필요하므로)
print(x.grad) 
y.backward(gradients) # 2번 미분해야하는 경우 retain_graph=True,여러번 호출 시, 미분 값이 더해져 중첩됨 
print(x.grad)

tensor([300.0000,   0.3000])
tensor([600.0000,   0.6000])


In [15]:
x = torch.randn(2, requires_grad=True)
y = x * 3
z = x / 2
w = x + y 
# 각기 다른 computation graph 가짐
print(w,y,z,sep='\n') 
print(w.grad_fn,y.grad_fn,z.grad_fn,sep='\n') 
# grad_fn attr을 통하여 호출할 backpropagation class를 알 수 있다

tensor([-0.4437, -1.3125], grad_fn=<AddBackward0>)
tensor([-0.3328, -0.9844], grad_fn=<MulBackward0>)
tensor([-0.0555, -0.1641], grad_fn=<DivBackward0>)


In [35]:
# CAM을 이용한 CNN visualization 등에서 중간층 Layer의 Gradient값이 필요할 때가 있다.
# hook을 사용하여 중간의 gradient 값을 저장해놓을 수 있다.
# hooking : 양 객체 사이의 통신내용(메세지, 값, 이벤트, gradient 등)을 가져오는 것

# register_hook(hook) : registers a backward hook
# register_forward_hook(hook) : registers a forward hook
class SimpleNet(nn.Module):# hook을 사용해보기위한 평범한 NN
  def __init__(self):
    super(SimpleNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, 5)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(10, 20, 5)
    self.pool2 = nn.MaxPool2d(2,2)
    self.fc = nn.Linear(320, 50)
    self.out = nn.Linear(50, 10)

  def forward(self, input):
    x = self.pool1(F.relu(self.conv1(input)))
    x = self.pool2(F.relu(self.conv2(x)))
    x = x.view(x.size(0),-1)
    x = F.relu(self.fc(x))
    x = F.relu(self.out(x))
    return x

In [36]:
# hook을 사용하기 위해...
# 1. Hook의 signature 정의
# hook이 실행됬을 때 해야할 활동들을 정의
# prototype에 맞게 해줘야함, self, input, output이 argument
def hook_func(self, input, output):
  # type(self) should be tensor
  print('Inside ' + self.__class__.__name__ + ' forward')
  print('')
  print('input: ', type(input))
  print('input[0]: ', type(input[0]))
  print('output: ', type(output))
  print('')


In [61]:
net = SimpleNet()
h = net.conv1.register_forward_hook(hook_func) # 1번째 layer에 forward 시 hook 실행 등록
print(h)
print(net.conv2.register_forward_hook(hook_func)) # 2번째 layer에forward 시 hook 실행 등록

In [62]:
input = torch.randn(1, 1, 28, 28)
out = net(input) # forward 시 hook function 실행되어 print 됨

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>



In [63]:
h.remove() # register 지우기, 1번째 layer의 register return(=handler)을 받아 remove() 함수로 지움
out = net(input) # 2번째 layer hook만 print됨

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>



In [46]:
# register_forward_hook(hook_pre) # layer의 forwar_pass 이전에 호줄되는 함수
def hook_pre(self, input, output):
  print('Inside ' + self.__class__.__name__ + ' forward')
  print('')
  print('input: ', type(input))
  print('input[0]: ', type(input[0]))

net = SimpleNet()
net.conv1.register_forward_hook(hook_pre) # layer의 forwar_pass 이전에 호줄되는 함수

input = torch.randn(1, 1, 28, 28)
out = net(input)

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>


In [58]:
# register_backward_hook(hook_grad) # layer의 backward_pass 이후에 호줄되는 함수
# register_full_backward_hook(hook_grad)# forward시 autograd nodes가 많이 포함될 경우 full 버전을 써야함 

def hook_grad(self, grad_input, grad_output): 
  # grad_input, grad_output을 내부 로직에서 바꾸면 안된다
  # 굳이 바꾸고 싶으면 return new_grad_input, new_grad_output
  # 새로운 input, output을 정해주고 return 해줘야함
  print('Inside ' + self.__class__.__name__ + ' backward')
  print('Inside classs:' + self.__class__.__name__)

  print('grad_input: ', type(grad_input))
  print('grad_input[0]: ', type(grad_input[0]))
  print('grad_output: ', type(grad_output))
  print('grad_output[0]: ', type(grad_output[0]))

net = SimpleNet()
net.conv1.register_backward_hook(hook_grad) # backward_pass 이전에 호줄되는 함수
# net.conv1.register_full_backward_hook(hook_grad) # forward시 autograd nodes가 많이 포함될 경우 full 버전을 써야함 

input = torch.randn(1, 1, 28, 28)
out = net(input)

target = torch.tensor([3], dtype=torch.long)
loss_fn = nn.CrossEntropyLoss()
err = loss_fn(out, target)
err.backward()

Inside Conv2d backward
Inside classs:Conv2d
grad_input:  <class 'tuple'>
grad_input[0]:  <class 'NoneType'>
grad_output:  <class 'tuple'>
grad_output[0]:  <class 'torch.Tensor'>


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
save_feat = [] # featuremap을 저장할 전역 변수 실행
def hook_feat(module, input, output): # hook function 정의
  save_feat.append(output)
  return output

# model layer를 돌면서 원하는 layer에 hook 등록
for name, module in model.get_model_shortcuts(): # model layer들을 가져오는 함수
  if(name == 'target_layer_name'):
    module.register_forward_hook(hook_feat)


img = img.unsqueeze(0)
s = model(img)[0]
print(save_feat)